In [1]:
import tensorflow as tf
import tensorflow.contrib.slim as slim
# from config import cfg
import numpy as np

/home/cn1lab005/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
iter_routing = 2
ac_lambda0 = 0.01, 
#'\lambda in the activation function a_c, iteration 0')
ac_lambda_step = 0.01,
#'It is described that \lambda increases at each iteration with a fixed schedule, however specific super parameters is absent.')
epsilon = 1e-9

#batch_size = 50
################################
A = 32 # , 'number of channels in output from ReLU Conv1')
B = 8 # , 'number of capsules in output from PrimaryCaps')
C = 16 #, 'number of channels in output from ConvCaps1')
D = 16 # , 'number of channels in output from ConvCaps2')
is_train = True
num_classes = 10

In [3]:
def kernel_tile(input, kernel, stride):
    # output = tf.extract_image_patches(input, ksizes=[1, kernel, kernel, 1], strides=[1, stride, stride, 1], rates=[1, 1, 1, 1], padding='VALID')

    input_shape = input.get_shape()
    tile_filter = np.zeros(shape=[kernel, kernel, input_shape[3],
                                  kernel * kernel], dtype=np.float32)
    for i in range(kernel):
        for j in range(kernel):
            tile_filter[i, j, :, i * kernel + j] = 1.0

    tile_filter_op = tf.constant(tile_filter, dtype=tf.float32)
    output = tf.nn.depthwise_conv2d(input, tile_filter_op, strides=[
                                    1, stride, stride, 1], padding='VALID')
    output_shape = output.get_shape()
    output = tf.reshape(output, shape=[-1, int( # -1== int(output_shape[0])
        output_shape[1]), int(output_shape[2]), int(input_shape[3]), kernel * kernel])
    print(output.get_shape(),"fdsggs")
    output = tf.transpose(output, perm=[0, 1, 2, 4, 3])

    return output

# input should be a tensor with size as [batch_size, caps_num_i, 16]
def mat_transform(input, caps_num_c, regularizer, bs):
    #batch_size = input.get_shape()[0]
    caps_num_i = int(input.get_shape()[1])
    output = tf.reshape(input, shape=[-1, caps_num_i, 1, 4, 4])# batch_size = -1
    # the output of capsule is miu, the mean of a Gaussian, and activation, the sum of probabilities
    # it has no relationship with the absolute values of w and votes
    # using weights with bigger stddev helps numerical stability
    w = slim.variable('w', shape=[1, caps_num_i, caps_num_c, 4, 4], dtype=tf.float32,
                      initializer=tf.truncated_normal_initializer(mean=0.0, stddev=1.0),
                      regularizer=regularizer)
    print("w",w.get_shape())
    with tf.variable_scope('tile___1'):
        w = tf.tile(w, [bs, 1, 1, 1, 1])
    print("w",w.get_shape())
    with tf.variable_scope('tile___2'):
        output = tf.tile(output, [1, 1, caps_num_c, 1, 1])
    with tf.variable_scope('tile___3'):
        k = tf.matmul(output, w)
        votes = tf.reshape(k, [-1, caps_num_i, caps_num_c, 16]) #batch_size = -1
    #votes = tf.reshape(tf.matmul(output, w), [batch_size, caps_num_i, caps_num_c, 16])

    return votes


In [4]:
def em_routing(votes, activation, caps_num_c, regularizer, r_in):
    test = []

    #batch_size = votes.get_shape()[0]
    caps_num_i = int(activation.get_shape()[1])
    n_channels = int(votes.get_shape()[-1])

    sigma_square = []
    miu = []
    activation_out = []
    beta_v = slim.variable('beta_v', shape=[caps_num_c, n_channels], dtype=tf.float32,
                           initializer=tf.constant_initializer(0.0),#tf.truncated_normal_initializer(mean=0.0, stddev=0.01),
                           regularizer=regularizer)
    beta_a = slim.variable('beta_a', shape=[caps_num_c], dtype=tf.float32,
                           initializer=tf.constant_initializer(0.0),#tf.truncated_normal_initializer(mean=0.0, stddev=0.01),
                           regularizer=regularizer)

    # votes_in = tf.stop_gradient(votes, name='stop_gradient_votes')
    # activation_in = tf.stop_gradient(activation, name='stop_gradient_activation')
    votes_in = votes
    print(votes_in,' = votes')
    activation_in = activation

    for iters in range(iter_routing):
        # if iters == cfg.iter_routing-1:

        # e-step
        if iters == 0:
            r = r_in# tf.constant(np.ones([batch_size, caps_num_i, caps_num_c], dtype=np.float32) / caps_num_c)
            print(r.get_shape(),"r shape__________")
        else:
            # Contributor: Yunzhi Shi
            # log and exp here provide higher numerical stability especially for bigger number of iterations
            log_p_c_h = -tf.log(tf.sqrt(sigma_square)) - \
                        (tf.square(votes_in - miu) / (2 * sigma_square))
            log_p_c_h = log_p_c_h - \
                        (tf.reduce_max(log_p_c_h, axis=[2, 3], keep_dims=True) - tf.log(10.0))
            p_c = tf.exp(tf.reduce_sum(log_p_c_h, axis=3))

            ap = p_c * tf.reshape(activation_out, shape=[-1, 1, caps_num_c]) # batch_size
            print(ap.get_shape(),"ap")
            # ap = tf.reshape(activation_out, shape=[batch_size, 1, caps_num_c])

            r = ap / (tf.reduce_sum(ap, axis=2, keepdims=True) + epsilon)

        # m-step
        r = r * activation_in
        r = r / (tf.reduce_sum(r, axis=2, keepdims=True)+epsilon)

        r_sum = tf.reduce_sum(r, axis=1, keepdims=True)
        r1 = tf.reshape(r / (r_sum + epsilon),
                        shape=[-1 , caps_num_i, caps_num_c, 1]) # batch_size
        print(r1.get_shape(),"r1")
        miu = tf.reduce_sum(votes_in * r1, axis=1, keepdims=True)
        sigma_square = tf.reduce_sum(tf.square(votes_in - miu) * r1,
                                     axis=1, keepdims=True) + epsilon

        if iters == iter_routing-1:
            r_sum = tf.reshape(r_sum, [-1, caps_num_c, 1])  # batch_size
            print(r_sum.get_shape(),"r_sum")
            cost_h = (beta_v + tf.log(tf.sqrt(tf.reshape(sigma_square,
                                                         shape=[-1, caps_num_c, n_channels])))) * r_sum
            print(cost_h.get_shape(),"cost_h") # batch_size
            activation_out = tf.nn.softmax(ac_lambda0 * (beta_a - tf.reduce_sum(cost_h, axis=2)))
        else:
            activation_out = tf.nn.softmax(r_sum)
        # if iters <= cfg.iter_routing-1:
        #     activation_out = tf.stop_gradient(activation_out, name='stop_gradient_activation')

    return miu, activation_out, test

In [5]:
def get_coord_add(dataset_name: str):
    import numpy as np
    # TODO: get coord add for cifar10/100 datasets (32x32x3)
    options = {'mnist': ([[[8., 8.], [12., 8.], [16., 8.]],
                          [[8., 12.], [12., 12.], [16., 12.]],
                          [[8., 16.], [12., 16.], [16., 16.]]], 28.),
               'smallNORB': ([[[8., 8.], [12., 8.], [16., 8.], [24., 8.]],
                              [[8., 12.], [12., 12.], [16., 12.], [24., 12.]],
                              [[8., 16.], [12., 16.], [16., 16.], [24., 16.]],
                              [[8., 24.], [12., 24.], [16., 24.], [24., 24.]]], 32.)
               }
    coord_add, scale = options[dataset_name]

    coord_add = np.array(coord_add, dtype=np.float32) / scale

    return coord_add

# Class for the Matrix Capsule

In [6]:
class MatrixCapsule(object):
    def __init__(self, is_train=True):
        tf.reset_default_graph()
        self.graph = tf.Graph()
        self._build_arch()
        self.sess = tf.Session()
        self.sess.run(tf.global_variables_initializer())
        print("Graph for Matrix Capsule is ready for training")
    def _build_arch(self):
        self.X = tf.placeholder(tf.float32, shape=(None, 28, 28, 1), name='X')
        #X = tf.placeholder(tf.float32, shape=(50, 28, 28, 1))
        self.Y = tf.placeholder(tf.float32, [None, 10], name='Y')
        # -------------------------------------------------------------------------------------
        self.bs = tf.placeholder(tf.int32, shape=(), name='bs')
        self.r_conv_caps1= tf.placeholder(tf.float32,[None, 72, C], name='r_conv_caps1') # 5*5*batch_size
        #r_conv_caps1 = tf.placeholder(tf.float32,[5*5*batch_size, 72, C]) # 5*5*batch_size

        self.r_conv_caps2 = tf.placeholder(tf.float32,[None, 144, D], name='r_conv_caps2') # 3*3*batch_size
        #r_conv_caps2 = tf.placeholder(tf.float32,[3*3*batch_size, 144, D]) # 3*3*batch_size

        self.r_class_caps = tf.placeholder(tf.float32,[None, 16, num_classes], name='r_class_caps') # 3*3*batch_size
        #r_class_caps = tf.placeholder(tf.float32,[3*3*batch_size, 16, num_classes]) # 3*3*batch_size

        self.coord_add_op_class_caps  = tf.placeholder(tf.float32,[None, 16, num_classes, 2], name='coord_add_op_class_caps')
                                                                         # 3*3*batch_size
        #coord_add_op_n  = tf.placeholder(tf.float32,[3*3*batch_size, 16, num_classes, 2]) # 3*3*batch_size
        # -------------------------------------------------------------------------------------
        data_size = int(self.X.get_shape()[1])
        # xavier initialization is necessary here to provide higher stability
        initializer = tf.truncated_normal_initializer(mean=0.0, stddev=0.01)
        # instead of initializing bias with constant 0, 
        # a truncated normal initializer is exploited here for higher stability 
        bias_initializer = tf.truncated_normal_initializer(mean=0.0, stddev=0.01)  # tf.constant_initializer(0.0)
        # The paper didnot mention any regularization, a common l2 regularizer to weights is added here
        weights_regularizer = tf.contrib.layers.l2_regularizer(5e-04)
        # weights_initializer=initializer,
        with slim.arg_scope([slim.conv2d], trainable=is_train, biases_initializer=bias_initializer, weights_regularizer=weights_regularizer):
            with tf.variable_scope('relu_conv1'):
                output = slim.conv2d(self.X, num_outputs=A, kernel_size=[5, 5], stride=2, padding='VALID', scope='relu_conv1', activation_fn=tf.nn.relu)
                data_size = int(np.floor((data_size - 4) / 2))
                #print(output.get_shape())
                #print(data_size)
                #assert output.get_shape() == [batch_size, data_size, data_size, 32]
                votes__1 = output
            with tf.variable_scope('primary_caps'):
                pose = slim.conv2d(output, num_outputs=B * 16,kernel_size=[1, 1], stride=1, padding='VALID', scope='primary_caps', activation_fn=None)
                activation = slim.conv2d(output, num_outputs=B, kernel_size=[
                                         1, 1], stride=1, padding='VALID', scope='primary_caps/activation', activation_fn=tf.nn.sigmoid)
                pose = tf.reshape(pose, shape=[-1, data_size, data_size, B, 16]) # (50, 12, 12, 8, 16)

                print(pose.get_shape())
                activation = tf.reshape(activation, shape=[-1, data_size, data_size, B, 1]) # (50, 12, 12, 8, 1)
                print(activation.get_shape())
                output = tf.concat([pose, activation], axis=4)
                output = tf.reshape(output, shape=[-1, data_size, data_size, B * 17]) # (50, 12, 12, 136)
                print(output.get_shape())

                #assert output.get_shape() == [batch_size, data_size, data_size, B * 17]
            with tf.variable_scope('conv_caps1') as scope:
                output = kernel_tile(output, 3, 2)
                data_size = int(np.floor((data_size - 2) / 2))
                print(data_size) # 5 
                output = tf.reshape(output, shape=[-1, 3 * 3 * B, 17]) 
                # batch_size * data_size * data_size  (1250, 72, 17) 
                print("1",output.get_shape())
                activation = tf.reshape(output[:, :, 16], shape=[-1, 3 * 3 * B, 1])
                print("output shape ---------------",output.get_shape())
                print("activation shape----------------------",activation.get_shape()) #  (1250, 72, 1)

                with tf.variable_scope('v') as scope:
                    votes = mat_transform(output[:, :, :16], C, weights_regularizer,
                                          bs = self.bs*data_size*data_size)
                    #bs*data_size*data_size)

                    print(votes.get_shape(),"votes shape")
                with tf.variable_scope('routing') as scope:
                    caps_num_i = int(activation.get_shape()[1])

                    miu, activation, _ = em_routing(votes, activation, C,
                                                    weights_regularizer,self.r_conv_caps1)
                    # miu, activation, _ = em_routing(votes, activation, C, weights_regularizer)
                    print("activation",activation.get_shape())
                pose = tf.reshape(miu, shape=[-1, data_size, data_size, C, 16])
                print("3",pose.get_shape()) # 50, 5, 5, 16, 16)
                activation = tf.reshape(activation, shape=[-1, data_size, data_size, C, 1])
                print("activation",activation.get_shape())
                cat_size =  activation.get_shape()[3]*activation.get_shape()[4] + pose.get_shape()[3] *pose.get_shape()[4]
                print(cat_size)
                output = tf.reshape(tf.concat([pose, activation], axis=4),[-1, data_size, data_size, cat_size])
                print("5",output.get_shape()) # (50, 5, 5, 272)

            with tf.variable_scope('conv_caps2') as scope:
                output = kernel_tile(output, 3, 1)
                data_size = int(np.floor((data_size - 2) / 1))
                output = tf.reshape(output, shape=[-1, 3 * 3 * C, 17]) # batch_size * data_size * data_size
                print("canv_caps2",output.get_shape(), data_size)
                activation = tf.reshape(output[:, :, 16], shape=[-1 , 3 * 3 * C, 1]) # batch_size * data_size * data_size
                print("canv_caps2_activation",activation.get_shape(), data_size)
                with tf.variable_scope('v') as scope:
                    votes = mat_transform(output[:, :, :16], D, weights_regularizer,
                                          bs = self.bs*data_size*data_size)
                    print(votes.get_shape(),"votes shape")
                with tf.variable_scope('routing') as scope:
                    caps_num_i = int(activation.get_shape()[1])
                    print(caps_num_i,"for 1")
                    miu, activation, _ = em_routing(votes, activation, D,
                                                    weights_regularizer, self.r_conv_caps2)
                pose = tf.reshape(miu, shape=[-1, D, 16]) # batch_size * data_size * data_size
                print("4",pose.get_shape())
                activation = tf.reshape(activation, shape=[-1, D, 1]) # batch_size * data_size * data_size
                print("4 ---activation",activation.get_shape())
            with tf.variable_scope('class_caps') as scope:
                with tf.variable_scope('v') as scope:
                    votes = mat_transform(pose, num_classes, weights_regularizer,
                                          bs = self.bs*data_size*data_size)
                    print(votes.get_shape(),"votes.getshape")
                    assert votes.get_shape()[1:] == [D, num_classes, 16]
                    '''coord_add = get_coord_add('mnist') 
                    coord_add = np.reshape(coord_add, newshape=[data_size * data_size, 1, 1, 2])
                    coord_add = np.tile(coord_add, [bs, D, num_classes, 1])
                    coord_add_op = tf.constant(coord_add, dtype=tf.float32)
                    print("___coord_add______",coord_add_op.shape)'''

                    votes = tf.concat([self.coord_add_op_class_caps, votes], axis=3)
                    print(votes.get_shape(),"coorr vote shape after  jnbfv")
                with tf.variable_scope('routing') as scope:
                    caps_num_i = int(activation.get_shape()[1])
                    print("_____",caps_num_i)
                    miu, activation, test2 = em_routing(votes, activation, num_classes,
                                                        weights_regularizer,self.r_class_caps)
                output = tf.reshape(activation, shape=[-1, data_size, data_size, num_classes]) #batch_size
                print("d op",output.get_shape())
            output = tf.reshape(tf.nn.avg_pool(output, ksize=[1, data_size, data_size, 1], strides=[
                        1, 1, 1, 1], padding='VALID'), shape=[-1, num_classes]) # batch_size
            print("miu  2",miu.get_shape())
            pose = tf.nn.avg_pool(tf.reshape(miu, shape=[-1, data_size, data_size,miu.get_shape()[2]*miu.get_shape()[3]
                                        ]), ksize=[1, data_size, data_size, 1], strides=[1, 1, 1, 1], padding='VALID')
            print("output_size_posssss",pose.get_shape())
            #miu  2 (450, 1, 10, 18)
            #output_size_posssss (50, 1, 1, 180)
            pose_out = tf.reshape(pose, shape=[-1, num_classes, 18])
            print("output_size",pose_out.get_shape())
            vector_j = tf.reshape(pose_out, shape= [-1, num_classes * 18])
            
        with tf.variable_scope('output_layer') as scope:
            self.logits = tf.contrib.layers.fully_connected(vector_j, num_outputs=10, activation_fn=None)
        print("shape of logits",self.logits)
        
        with tf.variable_scope('loss') as scope:
            self.cross_entropy_loss = tf.losses.softmax_cross_entropy(self.Y, self.logits)
            #print("output_size",self.logits.get_shape())

        with tf.variable_scope('optimizer') as scope:
            self.train_op = tf.train.AdamOptimizer(1e-3).minimize(self.cross_entropy_loss)

        with tf.variable_scope('prediction') as scope:  
            prediction = tf.nn.softmax(self.logits,-1)
            self.y_hat = tf.to_int32(tf.argmax(prediction, axis=1))

        with tf.variable_scope('acc'):
            labels = tf.to_int32(tf.argmax(self.Y, axis=1))
            correct_prediction = tf.equal(tf.to_int32(labels), self.y_hat)
            self.accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32)) * 100
            
    def train(self, batch_xs, batch_ys, batch_size_train):
        coord_add = get_coord_add('mnist') 
        coord_add = np.reshape(coord_add, newshape=[3 * 3, 1, 1, 2]) #data_size =3
        coord_add = np.tile(coord_add, [batch_size_train, D, num_classes, 1])
        r_conv_caps1_in = np.ones([5*5*batch_size_train, 72, C]) / C
        r_conv_caps2_in = np.ones([3*3*batch_size_train, 144, D]) / D
        r_class_caps_in = np.ones([3*3*batch_size_train, 16, num_classes]) / num_classes
        _, loss, acc = self.sess.run([self.train_op,self.cross_entropy_loss,self.accuracy],feed_dict={
                                        self.X: batch_xs,
                                        self.r_conv_caps1: r_conv_caps1_in,
                                        self.r_conv_caps2: r_conv_caps2_in,
                                        self.r_class_caps: r_class_caps_in,
                                        self.coord_add_op_class_caps: coord_add,
                                        self.bs: np.int32(batch_size_train),
                                        self.Y:batch_ys})
        return loss, acc
    def test_acc(self, batch_xs, batch_ys, batch_size_train):
        coord_add = get_coord_add('mnist') 
        coord_add = np.reshape(coord_add, newshape=[3 * 3, 1, 1, 2]) #data_size =3
        coord_add = np.tile(coord_add, [batch_size_train, D, num_classes, 1])
        r_conv_caps1_in = np.ones([5*5*batch_size_train, 72, C]) / C
        r_conv_caps2_in = np.ones([3*3*batch_size_train, 144, D]) / D
        r_class_caps_in = np.ones([3*3*batch_size_train, 16, num_classes]) / num_classes
        acc = self.sess.run(self.accuracy,feed_dict={
                                        self.X: batch_xs,
                                        self.r_conv_caps1: r_conv_caps1_in,
                                        self.r_conv_caps2: r_conv_caps2_in,
                                        self.r_class_caps: r_class_caps_in,
                                        self.coord_add_op_class_caps: coord_add,
                                        self.bs: np.int32(batch_size_train),
                                        self.Y:batch_ys})
        return acc

In [7]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [ ]:
model = MatrixCapsule()
print("session started")
batch_size_train= 50
batch_size_test = 30
n_itrs = 3000
for step in range(n_itrs):
    batch_xs, batch_ys = mnist.train.next_batch(batch_size_train)
    batch_xs = batch_xs.reshape(batch_size_train, 28, 28,1)
    batch_ys = batch_ys.reshape(batch_size_train, 10)
    loss, acc =  model.train(batch_xs, batch_ys, batch_size_train)
    assert not np.isnan(loss), 'Something wrong! loss is nan...'
    #print(step)
    if (step % 5 == 0):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size_test)
        batch_xs = batch_xs.reshape(batch_size_test, 28, 28,1)
        batch_ys = batch_ys.reshape(batch_size_test, 10)
        tac = model.test_acc(batch_xs, batch_ys, batch_size_test)
        print("step-{} summary: loss= {}, training accuracy = {}, test acc = {}".format(step, loss, acc, tac))
        

(?, 12, 12, 8, 16)
(?, 12, 12, 8, 1)
(?, 12, 12, 136)
(?, 5, 5, 136, 9) fdsggs
5
1 (?, 72, 17)
output shape --------------- (?, 72, 17)
activation shape---------------------- (?, 72, 1)
w (1, 72, 16, 4, 4)
w (?, 72, 16, 4, 4)
(?, 72, 16, 16) votes shape
Tensor("conv_caps1/v/tile___3/Reshape:0", shape=(?, 72, 16, 16), dtype=float32)  = votes
(?, 72, 16) r shape__________
(?, 72, 16, 1) r1
Instructions for updating:
keep_dims is deprecated, use keepdims instead
(?, 72, 16) ap
(?, 72, 16, 1) r1
(?, 16, 1) r_sum
(?, 16, 16) cost_h
activation (?, 16)
3 (?, 5, 5, 16, 16)
activation (?, 5, 5, 16, 1)
272
5 (?, 5, 5, 272)
(?, 3, 3, 272, 9) fdsggs
canv_caps2 (?, 144, 17) 3
canv_caps2_activation (?, 144, 1) 3
w (1, 144, 16, 4, 4)
w (?, 144, 16, 4, 4)
(?, 144, 16, 16) votes shape
144 for 1
Tensor("conv_caps2/v/tile___3/Reshape:0", shape=(?, 144, 16, 16), dtype=float32)  = votes
(?, 144, 16) r shape__________
(?, 144, 16, 1) r1
(?, 144, 16) ap
(?, 144, 16, 1) r1
(?, 16, 1) r_sum
(?, 16, 16) cost_h


step-315 summary: loss= 1.9006478786468506, training accuracy = 32.0, test acc = 26.666667938232422
step-320 summary: loss= 2.1093108654022217, training accuracy = 18.0, test acc = 36.66666793823242
step-325 summary: loss= 2.0133211612701416, training accuracy = 40.0, test acc = 33.333335876464844
step-330 summary: loss= 2.1492726802825928, training accuracy = 20.0, test acc = 33.333335876464844
step-335 summary: loss= 1.9389210939407349, training accuracy = 22.0, test acc = 36.66666793823242
step-340 summary: loss= 1.9050756692886353, training accuracy = 34.0, test acc = 33.333335876464844
step-345 summary: loss= 2.114236831665039, training accuracy = 26.0, test acc = 30.000001907348633
step-350 summary: loss= 2.024559259414673, training accuracy = 36.0, test acc = 30.000001907348633
step-355 summary: loss= 1.9313664436340332, training accuracy = 40.0, test acc = 43.33333206176758
step-360 summary: loss= 2.0715723037719727, training accuracy = 24.0, test acc = 20.0
step-365 summary: l

step-745 summary: loss= 1.0752816200256348, training accuracy = 70.0, test acc = 76.66666412353516
step-750 summary: loss= 0.9351963996887207, training accuracy = 76.0, test acc = 63.33333206176758
step-755 summary: loss= 0.9968814253807068, training accuracy = 68.0, test acc = 70.0
step-760 summary: loss= 0.8309645652770996, training accuracy = 80.0, test acc = 73.33333587646484
step-765 summary: loss= 1.0568162202835083, training accuracy = 66.0, test acc = 76.66666412353516
step-770 summary: loss= 1.02456796169281, training accuracy = 68.0, test acc = 63.33333206176758
step-775 summary: loss= 1.1860712766647339, training accuracy = 58.0, test acc = 76.66666412353516
step-780 summary: loss= 1.0909037590026855, training accuracy = 60.000003814697266, test acc = 76.66666412353516
step-785 summary: loss= 0.798025369644165, training accuracy = 92.0, test acc = 80.0
step-790 summary: loss= 0.8997924327850342, training accuracy = 80.0, test acc = 86.66666412353516
step-795 summary: loss= 1

step-1175 summary: loss= 0.6684384346008301, training accuracy = 72.0, test acc = 90.0
step-1180 summary: loss= 0.39449751377105713, training accuracy = 88.0, test acc = 90.0
step-1185 summary: loss= 0.3045433759689331, training accuracy = 96.0, test acc = 93.33333587646484
step-1190 summary: loss= 0.3401741683483124, training accuracy = 86.0, test acc = 73.33333587646484
step-1195 summary: loss= 0.3670032024383545, training accuracy = 86.0, test acc = 80.0
step-1200 summary: loss= 0.6861773133277893, training accuracy = 82.0, test acc = 86.66666412353516
step-1205 summary: loss= 0.6039700508117676, training accuracy = 88.0, test acc = 90.0
step-1210 summary: loss= 0.5501613020896912, training accuracy = 78.0, test acc = 90.0
step-1215 summary: loss= 0.4578954577445984, training accuracy = 86.0, test acc = 86.66666412353516
step-1220 summary: loss= 0.362844318151474, training accuracy = 92.0, test acc = 90.0
step-1225 summary: loss= 0.3221363127231598, training accuracy = 94.0, test ac

step-1605 summary: loss= 0.4545876979827881, training accuracy = 86.0, test acc = 90.0
step-1610 summary: loss= 0.3651581108570099, training accuracy = 90.0, test acc = 86.66666412353516
step-1615 summary: loss= 0.47842612862586975, training accuracy = 84.0, test acc = 90.0
step-1620 summary: loss= 0.3884197175502777, training accuracy = 86.0, test acc = 93.33333587646484
step-1625 summary: loss= 0.23159103095531464, training accuracy = 88.0, test acc = 93.33333587646484
step-1630 summary: loss= 0.3055935204029083, training accuracy = 96.0, test acc = 96.66666412353516
step-1635 summary: loss= 0.35574501752853394, training accuracy = 92.0, test acc = 96.66666412353516
step-1640 summary: loss= 0.2539653480052948, training accuracy = 90.0, test acc = 93.33333587646484
step-1645 summary: loss= 0.2545824944972992, training accuracy = 96.0, test acc = 86.66666412353516
step-1650 summary: loss= 0.6332545280456543, training accuracy = 82.0, test acc = 93.33333587646484
step-1655 summary: loss

step-2020 summary: loss= 0.22325879335403442, training accuracy = 94.0, test acc = 83.33332824707031
step-2025 summary: loss= 0.19846881926059723, training accuracy = 92.0, test acc = 83.33332824707031
step-2030 summary: loss= 0.31420233845710754, training accuracy = 96.0, test acc = 93.33333587646484
step-2035 summary: loss= 0.3402603268623352, training accuracy = 88.0, test acc = 96.66666412353516
step-2040 summary: loss= 0.30355948209762573, training accuracy = 90.0, test acc = 90.0
step-2045 summary: loss= 0.15601210296154022, training accuracy = 96.0, test acc = 100.0
step-2050 summary: loss= 0.2593457102775574, training accuracy = 92.0, test acc = 93.33333587646484
step-2055 summary: loss= 0.2946973443031311, training accuracy = 90.0, test acc = 96.66666412353516
step-2060 summary: loss= 0.33413010835647583, training accuracy = 90.0, test acc = 96.66666412353516
step-2065 summary: loss= 0.17819887399673462, training accuracy = 96.0, test acc = 93.33333587646484
step-2070 summary: